<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Learning Agency Lab - Automated Essay Scoring 2.0

- Môn học: Phân tích dữ liệu thông minh
- Nhóm: 10

# **THỬ NGHIỆM TRÊN MÔ HÌNH TỰ XÂY DỰNG**

##  **A. Tiền xử lý dữ liệu**


### **1. Import các thư viện cần thiết**

- Cài đặt thư viện cần thiết.

In [ ]:
!pip install pyspellchecker
!pip install catboost

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import time
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from sklearn.svm import SVR
import tensorflow as tf
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
import lightgbm as lgb

from spellchecker import SpellChecker
import warnings
warnings.filterwarnings('ignore')

### **2. Đọc dữ liệu**
- Để dễ dàng đồng bộ đường dẫn, nhóm sẽ thực hiện download file csv trực tiếp từ Google Drive.

In [ ]:
# URL tải xuống trực tiếp của tệp CSV trên Google Drive
TRAIN_ID = '1hUhF4f-gGTixo_-b-ytez01_swNBslIG'
url = f"https://drive.google.com/uc?export=download&id={TRAIN_ID}"
# Đọc tệp CSV từ URL
try:
    train = pd.read_csv(url)
    display(train.head())
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [ ]:
TEST_ID = '1kJa0kIeP0RpAFFcKa1QpFP7o4xtpxjet'
url = f"https://drive.google.com/uc?export=download&id={TEST_ID}"
# Đọc tệp CSV từ URL
try:
    test = pd.read_csv(url)
    display(test.head())
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


### **3. Tiền xử lý dữ liệu**

Cần **làm sạch văn bản**, nhằm chuẩn hóa và loại bỏ những thành phần không cần thiết trước khi tiến hành các bước xử lý tiếp theo.

- Văn bản được chuyển đổi toàn bộ về **chữ thường** để đảm bảo tính nhất quán và tránh phân biệt giữa chữ hoa và chữ thường.
- Các **thẻ HTML**, thẻ tên người dùng (bắt đầu bằng @), **hashtag** (bắt đầu bằng #), và đường dẫn **URL** đều được loại bỏ để giữ lại nội dung văn bản thực sự.
- Các **ký tự đặc biệt** và các **số** trong văn bản, thường không mang lại giá trị ngữ nghĩa, cũng được loại bỏ.
- Các **dấu câu liên tiếp** được xử lý và thay thế bằng một ký tự duy nhất.
- Các **từ viết tắt** được mở rộng thành dạng đầy đủ để đảm bảo tính nhất quán. Tham khảo từ: [Expand Contractions](https://www.kaggle.com/code/xianhellg/more-feature-engineering-feature-selection-0-817?scriptVersionId=173223907&cellId=11)

In [ ]:
def expand_contractions(text):
    contractions_dict = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not","oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have","that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have","there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
    "to've": "to have","wasn't": "was not","weren't": "were not",
    "we'd": "we had",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "what'll": "what will","what'll've": "what will have","what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have",
    "where'd": "where did","where's": "where is","where've": "where have",
    "who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
    "will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
    "y'all": "you all","y'alls": "you alls","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
    "y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have",
    "you're": "you are",  "you've": "you have"
    }
    contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

    return contractions_re.sub(lambda match: contractions_dict[match.group(0)], text)

def clean_text(text):
    text = text.lower()
    text = re.compile(r'<.*?>').sub(r'', text)
    text = re.sub(r'@\w+\s*', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = text.replace(u'\xa0', ' ')
    text = re.sub(r'\d+', '', text)
    text = expand_contractions(text)
    text = re.sub(r'\.+', '.', text)
    text = re.sub(r'\,+', ',', text)
    text = text.strip()

    return text

train['full_text'] = train['full_text'].apply(clean_text)
test['full_text'] = test['full_text'].apply(clean_text)

### **4. Feature engineering**

#### **Rút trích đặc trưng đoạn văn**

Phân tích về đặc trưng đoạn văn giúp hiểu rõ hơn về cấu trúc văn bản, độ phức tạp và ngữ cảnh văn bản.
- Độ dài đoạn văn phản ánh mức độ chi tiết và độ phức tạp của văn bản.
- Số câu trong đoạn văn giúp xác định mức độ chi tiết và cách trình bày của văn bản.
- Số từ trong đoạn văn là một chỉ số quan trọng để đo lường độ phức tạp và độ dài của văn bản.
- Lấy giá trị đầu tiên và cuối cùng của mỗi đặc trưng  giúp hiểu thêm về sự biến đổi và xu hướng của các đoạn văn trong văn bản.


In [ ]:
def extract_paragraph_features(data):
    data['paragraph'] = data['full_text'].str.split('\n\n')

    data = data.explode('paragraph')
    data = data[data['paragraph'].str.strip() != ""]
    data['paragraph'] = data['paragraph'].apply(clean_text)
    data['paragraph_length'] = data['paragraph'].apply(len)
    data['sentence_count'] = data['paragraph'].apply(lambda x: len(x.split('.')))
    data['word_count'] = data['paragraph'].apply(lambda x: len(x.split()))
    data['fullstop_ratio'] = data.apply(lambda row: row['paragraph'].count('.') / len(row['paragraph']) if len(row['paragraph']) > 0 else 0, axis=1)

    features = ['paragraph_length', 'sentence_count', 'word_count']

    def calculate_aggregations(group):
        aggs = {}
        for feat in features:
            aggs[f'{feat}_max'] = group[feat].max()
            aggs[f'{feat}_mean'] = group[feat].mean()
            aggs[f'{feat}_min'] = group[feat].min()
            aggs[f'{feat}_first'] = group[feat].iloc[0]
            aggs[f'{feat}_last'] = group[feat].iloc[-1]
            aggs[f'{feat}_sum'] = group[feat].sum()
            aggs[f'{feat}_q1'] = group[feat].quantile(0.25)
            aggs[f'{feat}_med'] = group[feat].median()
            aggs[f'{feat}_q3'] = group[feat].quantile(0.75)

        length_counts = {f'length_ge_{i}_count': (group['paragraph_length'] >= i).sum() for i in [50, 75, 100, 125, 150, 175, 200, 250, 300, 350, 400, 500, 600, 700]}
        length_counts.update({f'length_le_{i}_count': (group['paragraph_length'] <= i).sum() for i in [25, 49]})

        aggs.update(length_counts)

        return pd.Series(aggs)

    data = data.groupby('essay_id').apply(calculate_aggregations).reset_index()

    return data

train_feats = extract_paragraph_features(train)
test_feats = extract_paragraph_features(test)

feature_names = [col for col in train_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in train set: ', len(feature_names))

feature_names = [col for col in test_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  43
Feature count in test set:  43


#### **Rút trích đặc trưng câu văn**

Tương tự, việc phân tích về đặc trưng câu văn cũng giúp hiểu rõ hơn về cấu trúc, độ phức tạp và ngữ cảnh văn bản.

- Độ dài câu văn phản ánh mức độ chi tiết của thông tin và độ phức tạp trong cấu trúc câu.
- Giá trị đầu tiên và cuối cùng của mỗi đặc trưng giúp hiểu thêm về sự biến đổi và xu hướng của các câu văn trong văn bản.
- Giá trị phân vị của các đặc trưng phản ánh mức độ phân tán và sự biến động trong độ dài và số lượng từ của các câu văn.

In [ ]:
def extract_sentence_features(data):
    data['sentence'] = data['full_text'].apply(clean_text).str.split('.')
    data = data.explode('sentence')
    data['sentence_len'] = data['sentence'].apply(len)
    data = data[data['sentence_len'] >= 15]
    data['sentence_word_count'] = data['sentence'].apply(lambda x: len(x.split()))
    data['comma_ratio'] = data['sentence'].apply(lambda x: x.count(',') / len(x) if len(x) > 0 else 0)

    features = ['sentence_len', 'sentence_word_count']

    def calculate_aggregations(group):
        aggs = {}
        for feat in features:
            aggs[f'{feat}_max'] = group[feat].max()
            aggs[f'{feat}_mean'] = group[feat].mean()
            aggs[f'{feat}_min'] = group[feat].min()
            aggs[f'{feat}_first'] = group[feat].iloc[0]
            aggs[f'{feat}_last'] = group[feat].iloc[-1]
            aggs[f'{feat}_sum'] = group[feat].sum()
            aggs[f'{feat}_q1'] = group[feat].quantile(0.25)
            aggs[f'{feat}_med'] = group[feat].median()
            aggs[f'{feat}_q3'] = group[feat].quantile(0.75)

        sentence_length_counts = {f'sentence_length_ge_{i}_count': (group['sentence_len'] >= i).sum() for i in [15, 50, 100, 150, 200, 250, 300]}

        aggs.update(sentence_length_counts)

        return pd.Series(aggs)

    data = data.groupby('essay_id').apply(calculate_aggregations).reset_index()

    return data

sentence_train = extract_sentence_features(train)
sentence_test = extract_sentence_features(test)

train_feats = train_feats.merge(sentence_train, on='essay_id', how='left')
test_feats = test_feats.merge(sentence_test, on='essay_id', how='left')

feature_names = [col for col in train_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in train set: ', len(feature_names))

feature_names = [col for col in test_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  68
Feature count in test set:  68


#### **Rút trích đặc trưng từ**

- Độ dài từ phản ánh mức độ phức tạp, tính học thuật và mức độ thông tin của từ vựng sử dụng trong văn bản.
- Giá trị lớn nhất, trung bình, độ lệch chuẩn, và các phần tư cung cấp cái nhìn tổng quát về sự phân bố độ dài từ trong văn bản.
- Số lượng lỗi chính tả cho biết về chất lượng từ vựng và khả năng biểu đạt của văn bản.

In [ ]:
def extract_word_features(data):
    spellchecker = SpellChecker()
    data['word'] = data['full_text'].apply(clean_text).str.split(' ')
    data = data.explode('word')
    data['word_len'] = data['word'].apply(len)
    data = data[data['word_len'] != 0]

    # Thêm cột 'spelling_error' để xác định lỗi chính tả
    def is_spelling_error(word):
        return word in spellchecker.unknown([word])

    data['is_spelling_error'] = data['word'].apply(is_spelling_error)

    def calculate_aggregations(group):
        aggs = {}

        feat = 'word_len'
        aggs[f'{feat}_max'] = group[feat].max()
        aggs[f'{feat}_mean'] = group[feat].mean()
        aggs[f'{feat}_min'] = group[feat].min()
        aggs[f'{feat}_first'] = group[feat].iloc[0]
        aggs[f'{feat}_last'] = group[feat].iloc[-1]
        aggs[f'{feat}_sum'] = group[feat].sum()
        aggs[f'{feat}_q1'] = group[feat].quantile(0.25)
        aggs[f'{feat}_med'] = group[feat].median()
        aggs[f'{feat}_q3'] = group[feat].quantile(0.75)

        aggs['spelling_errors_count'] = group['is_spelling_error'].sum()

        word_length_counts = {f'word_length_ge_{i+1}_count': (group['word_len'] >= i+1).sum() for i in range(15)}
        aggs.update(word_length_counts)

        return pd.Series(aggs)

    data = data.groupby('essay_id').apply(calculate_aggregations).reset_index()

    return data

word_train = extract_word_features(train)
word_test = extract_word_features(test)

train_feats = train_feats.merge(word_train, on='essay_id', how='left')
test_feats = test_feats.merge(word_test, on='essay_id', how='left')

feature_names = [col for col in train_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in train set: ', len(feature_names))

feature_names = [col for col in test_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  93
Feature count in test set:  93


#### **Phương pháp bag-of-n-gram**

Chúng ta sẽ mã hoá các cụm n-grams trong câu thành một vector có độ dài bằng số lượng các n-grams trong từ điển và đếm tần suất xuất hiện của các cụm đó. Như vậy thì mỗi cụm n-grams sẽ trở thành một chiều biểu diễn trong không gian của vector đầu ra.

In [ ]:
vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,3),
            min_df=0.10,
            max_df=0.85,
)

train_cnt = vectorizer_cnt.fit_transform([i for i in train['full_text']])
test_cnt = vectorizer_cnt.transform([i for i in test['full_text']])

cnt_train = pd.DataFrame(train_cnt.toarray())
cnt_test = pd.DataFrame(test_cnt.toarray())

cnt_train.columns = [f'cnt_{i}' for i in range(len(cnt_train.columns))]
cnt_test.columns = [f'cnt_{i}' for i in range(len(cnt_test.columns))]
cnt_train['essay_id'] = train_feats['essay_id']
cnt_test['essay_id'] = test_feats['essay_id']

train_feats = train_feats.merge(cnt_train, on='essay_id', how='left')
test_feats = test_feats.merge(cnt_test, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Feature count in train set: ', len(feature_names))

feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], test_feats.columns))
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  2088
Feature count in test set:  2088


#### **Phương pháp TF-IDF (Term Frequency, Inverse Document Frequency)**

TF-IDF là viết tắt của “Term Frequency, Inverse Document Frequency” - tạm dịch “Tần suất thuật ngữ, Tần suất tài liệu nghịch đảo”. Đó là một cách để chấm điểm tầm quan trọng của các từ (hoặc \"các thuật ngữ\") dựa trên tần suất xuất hiện của chúng xuất hiện trên nhiều tài liệu dựa trên quy tắc sau:
- Nếu một từ xuất hiện thường xuyên trong tài liệu, điều đó rất quan trọng $\Rightarrow$ cho từ này điểm cao.
- Nhưng nếu một từ xuất hiện trong nhiều tài liệu, thì đó không phải là mã định danh duy nhất $\Rightarrow$ cho từ đó điểm thấp.

Do đó, những từ phổ biến như `the` và `for` xuất hiện trong nhiều tài liệu sẽ được scaled down. Các từ xuất hiện thường xuyên trong một tài liệu sẽ được scaled up.

Với những giải thích trên, ta có công thức tính trọng số của một từ trong tài liệu trong ngữ liệu như sau:
$$w_{i,j} = tf_{i,j} \cdot idf_i = tf_{i,j} \cdot log(\frac {N}{df_i})$$

Trong đó:
- $tf_{i,j}$: Tần suất xuất hiện của i trong j
- $N$: Tổng số tài liệu
- $df_i$: Số tài liệu chứa i

Tham khảo: [An Introduction to TF-IDF using Python](https://medium.com/analytics-vidhya/an-introduction-to-tf-idf-using-python-5f9d1a343f77)

In [ ]:
vectorizer_tfidf = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(2,3),
    min_df=0.05,
    max_df=0.95,
    sublinear_tf=True,
)

train_tfidf = vectorizer_tfidf.fit_transform([i for i in train['full_text']])
test_tfidf = vectorizer_tfidf.transform([i for i in test['full_text']])

tfidf_train = pd.DataFrame(train_tfidf.toarray())
tfidf_test = pd.DataFrame(test_tfidf.toarray())

tfidf_train.columns = [f'tfid_{i}' for i in range(len(tfidf_train.columns))]
tfidf_test.columns = [f'tfid_{i}' for i in range(len(tfidf_test.columns))]
tfidf_train['essay_id'] = train_feats['essay_id']
tfidf_test['essay_id'] = test_feats['essay_id']

train_feats = train_feats.merge(tfidf_train, on='essay_id', how='left')
test_feats = test_feats.merge(tfidf_test, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Feature count in train set: ', len(feature_names))

feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], test_feats.columns))
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  4975
Feature count in test set:  4975


## **B. Xây dựng mô hình**

### **1. Chuẩn bị dữ liệu**

#### **Cài đặt kiểm tra chéo dữ liệu với StratifiedKFold**

In [ ]:
K = 5
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
train_feats = train_feats.merge(train, on='essay_id', how='left')

for i, (_, val_index) in enumerate(skf.split(train_feats, train_feats['score'])):
    train_feats.loc[val_index, 'fold'] = i

#### **Feature selection**

In [ ]:
target = ['score']
drop_columns = ['essay_id', 'fold', 'full_text', 'paragraph', 'sentence', 'word']

#### **Định nghĩa hàm đánh giá và hàm mục tiêu**

- Sử dụng **mô hình hồi quy** giúp cho việc học dữ liệu không bị overfitting. Vì thế vấn đề xảy ra khi sử dụng bài toán hồi quy cho biến dự đoán phân loại có thứ tự là xác định đúng được làm tròn số ở ngưỡng nào.
- Giá trị nhãn được **cộng thêm giá trị trung bình** của y nhằm điều chỉnh các nhãn về trung tâm của phân phối dữ liệu, làm giảm sai lệch và giúp các nhãn có giá trị gần với trung bình của tập dữ liệu gốc.
- Giá trị nhãn dự đoán được **giới hạn nằm trong khoảng từ 1 đến 6**, giả định rằng nhãn có giá trị trong khoảng này. Điều này giúp giữ các nhãn dự đoán trong phạm vi hợp lý và tránh các giá trị bất thường.


In [ ]:
# Hàm tính toán QWK
def calculate_quadratic_weighted_kappa(y_true, y_pred):
    y_true_adjusted = (y_true + a).round()
    y_pred_adjusted = (y_pred + a).clip(1, 6).round()
    qwk_score = cohen_kappa_score(y_true_adjusted, y_pred_adjusted, weights="quadratic")
    return 'QWK', qwk_score, True

# Hàm mục tiêu cho QWK
def qwk_objective(y_true, y_pred):
    y_true_adjusted = y_true + a
    y_pred_adjusted = y_pred + a
    y_pred_adjusted = y_pred_adjusted.clip(1, 6)

    f = 1 / 2 * np.sum((y_pred_adjusted - y_true_adjusted) ** 2)
    g = 1 / 2 * np.sum((y_pred_adjusted - a) ** 2 + b)

    df = y_pred_adjusted - y_true_adjusted
    dg = y_pred_adjusted - a

    grad = (df / g - f * dg / g ** 2) * len(y_true_adjusted)
    hess = np.ones(len(y_true_adjusted))
    return grad, hess

# Hàm tính toán các tham số cho QWK
def calculate_qwk_parameters(y):
    mean_value = y.mean()
    variance_value = (y ** 2).mean() - mean_value ** 2
    return np.round(mean_value, 4), np.round(variance_value, 4)

### **2. Huấn luyện mô hình**

#### **Mô hình Support Vector Regression**



In [ ]:
score_qwk_list = []
predictions, actuals = [], []
t0 = time.time()

for fold in range(K):
    print('### Fold', fold+1)

    a, b = calculate_qwk_parameters(train_feats[train_feats['fold'] != fold]['score'])
    X_train = train_feats[train_feats["fold"] != fold].drop(columns=drop_columns+target)
    y_train = train_feats[train_feats["fold"] != fold]['score'] - a
    X_valid = train_feats[train_feats["fold"] == fold].drop(columns=drop_columns+target)
    y_valid = train_feats[train_feats["fold"] == fold]['score'] - a

    model = SVR(C=10)
    model.fit(X_train, y_train)
    train_preds = model.predict(X_valid)

    actuals.extend(y_valid + a)
    predictions.extend(np.round(train_preds + a, 0))

    score_qwk_list.append(calculate_quadratic_weighted_kappa(y_valid, train_preds)[1])
    print(f"QWK score: {score_qwk_list[-1]}")
    print()

validation_score = cohen_kappa_score(actuals, predictions, weights="quadratic")
print(f"Validation score: {validation_score}")
print(f"Training time: {time.time() - t0}")

### Fold 1
QWK score: 0.7338583847037339

### Fold 2
QWK score: 0.7341843170448591

### Fold 3
QWK score: 0.7159574489948541

### Fold 4
QWK score: 0.7305887061497378

### Fold 5
QWK score: 0.7354530299979286

Validation score: 0.7300386962270629
Training time: 4875.6597468853


#### **Mô hình Neural Network**

In [ ]:
predictions, actuals = [], []
t0 = time.time()

for fold in range(5):
    a, b = calculate_qwk_parameters(train_feats[train_feats['fold'] != fold]['score'])
    train_data = train_feats[train_feats['fold'] != fold]
    val_data = train_feats[train_feats['fold'] == fold]

    X_train_tf = train_data.drop(columns=drop_columns+target).values
    y_train_tf = train_data['score'].values - a
    X_val_tf = val_data.drop(columns=drop_columns+target).values
    y_val_tf = val_data['score'].values - a

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                  loss='mean_squared_error')

    class QWKCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            y_val_pred = self.model.predict(X_val_tf).flatten()
            qwk_score = calculate_quadratic_weighted_kappa(y_val_tf, y_val_pred)
            print(f"Epoch {epoch + 1}: {qwk_score}")

    model.fit(X_train_tf, y_train_tf, epochs=20, batch_size=80, verbose=0, callbacks=[QWKCallback()])

    y_val_pred = model.predict(X_val_tf).flatten()
    actuals.extend(y_val_tf + a)
    predictions.extend(np.round(y_val_pred + a, 0))
    val_loss = model.evaluate(X_val_tf, y_val_tf, verbose=0)
    qwk_score = calculate_quadratic_weighted_kappa(y_val_tf, y_val_pred)

    print(f"Fold {fold}: Validation Loss = {val_loss}, QWK = {qwk_score[1]}")

validation_score = cohen_kappa_score(actuals, predictions, weights="quadratic")
print(f"Validation score: {validation_score}")
print(f"Training time: {time.time() - t0}")

109/109 [==============================] - 0s 3ms/step
Epoch 1: ('QWK', 0.3143398273010587, True)
109/109 [==============================] - 0s 3ms/step
Epoch 2: ('QWK', 0.39423227824468166, True)
109/109 [==============================] - 1s 5ms/step
Epoch 3: ('QWK', 0.7302142433378498, True)
109/109 [==============================] - 0s 3ms/step
Epoch 4: ('QWK', 0.4534452272102213, True)
109/109 [==============================] - 0s 3ms/step
Epoch 5: ('QWK', 0.5507850667502678, True)
109/109 [==============================] - 0s 3ms/step
Epoch 6: ('QWK', 0.660026591097352, True)
109/109 [==============================] - 0s 3ms/step
Epoch 7: ('QWK', 0.6509689532233327, True)
109/109 [==============================] - 0s 3ms/step
Epoch 8: ('QWK', 0.7273065506428212, True)
109/109 [==============================] - 0s 3ms/step
Epoch 9: ('QWK', 0.7358906995413061, True)
109/109 [==============================] - 0s 3ms/step
Epoch 10: ('QWK', 0.7813101113633021, True)
109/109 [==========

#### **Mô hình Cat Boost Classifier**

In [ ]:
t0 = time.time()
for fold in range(K):
    model = CatBoostRegressor(
                learning_rate=0.05,
                depth=5,
                l2_leaf_reg=0.1,
                iterations=700,
                random_seed=42,
                verbose=100)

    a, b = calculate_qwk_parameters(train_feats[train_feats['fold'] != fold]['score'])

    X_train = train_feats[train_feats['fold'] != fold].drop(columns=drop_columns+target)
    y_train = train_feats[train_feats['fold'] != fold]['score'] - a

    X_eval = train_feats[train_feats['fold'] == fold].drop(columns=drop_columns+target)
    y_eval = train_feats[train_feats['fold'] == fold]['score'] - a

    print(f"\nTraining fold {fold} with a: {a}, b: {b}")

    train_pool = Pool(X_train, y_train)
    eval_pool = Pool(X_eval, y_eval)

    model.fit(
        train_pool,
        eval_set=eval_pool,
        use_best_model=True
    )

    preds = model.predict(X_eval)

    actuals.extend(y_eval + a)
    predictions.extend(np.round(preds + a, 0))

validation_score = cohen_kappa_score(actuals, predictions, weights="quadratic")
print(f"Validation score: {validation_score}")
print(f"Training time: {time.time() - t0}")


Training fold 0 with a: 2.9484, b: 1.0917
0:	learn: 1.0199271	test: 1.0197014	best: 1.0197014 (0)	total: 900ms	remaining: 10m 28s
100:	learn: 0.6145596	test: 0.6252634	best: 0.6252634 (100)	total: 1m 10s	remaining: 7m 1s
200:	learn: 0.5663513	test: 0.5967255	best: 0.5967255 (200)	total: 2m 20s	remaining: 5m 48s
300:	learn: 0.5303362	test: 0.5830543	best: 0.5830543 (300)	total: 3m 28s	remaining: 4m 36s
400:	learn: 0.5035500	test: 0.5774621	best: 0.5774621 (400)	total: 4m 37s	remaining: 3m 26s
500:	learn: 0.4811673	test: 0.5744201	best: 0.5744201 (500)	total: 5m 44s	remaining: 2m 16s
600:	learn: 0.4613411	test: 0.5723723	best: 0.5723723 (600)	total: 6m 53s	remaining: 1m 8s
699:	learn: 0.4438615	test: 0.5715592	best: 0.5715592 (699)	total: 7m 59s	remaining: 0us

bestTest = 0.5715591723
bestIteration = 699


Training fold 1 with a: 2.9482, b: 1.0914
0:	learn: 1.0199056	test: 1.0206071	best: 1.0206071 (0)	total: 1.13s	remaining: 13m 10s
100:	learn: 0.6153167	test: 0.6231751	best: 0.6231751

#### **Mô hình Extreme Gradient Boosting (XGBoost)**

In [ ]:
t0 = time.time()
score_qwk_list = []
predictions, actuals = [], []

for fold in range(K):
    print('### Fold', fold+1)
    a, b = calculate_qwk_parameters(train_feats[train_feats['fold'] != fold]['score'])
    X_train = train_feats[train_feats["fold"] != fold].drop(columns=drop_columns+target)
    y_train = train_feats[train_feats["fold"] != fold]['score'] - a
    X_valid = train_feats[train_feats["fold"] == fold].drop(columns=drop_columns+target)
    y_valid = train_feats[train_feats["fold"] == fold]['score'] - a

    model = XGBRegressor(objective = qwk_objective)
    model.fit(X_train, y_train)

    train_preds = model.predict(X_valid)
    actuals.extend(y_valid + a)
    predictions.extend(np.round(train_preds + a, 0))

    score_qwk_list.append(calculate_quadratic_weighted_kappa(y_valid, train_preds)[1])
    print(f"QWK score: {score_qwk_list[-1]}")
    print()

validation_score = cohen_kappa_score(actuals, predictions, weights="quadratic")
print(f"Validation score: {validation_score}")
print(f"Training time: {time.time() - t0}")

### Fold 1
QWK score: 0.7749795785719933

### Fold 2
QWK score: 0.7714615240507813

### Fold 3
QWK score: 0.7600160225245447

### Fold 4
QWK score: 0.7620199798308096

### Fold 5
QWK score: 0.7595772434707013

Validation score: 0.7655090501505399
Training time: 962.5203611850739


#### **Mô hình Light Gradient-Boosting Machine**


In [ ]:
t0 = time.time()
lgb_models = []
predictions, actuals = [], []

training_callbacks = [
    lgb.log_evaluation(period=25),
    lgb.early_stopping(stopping_rounds=75, first_metric_only=True)
]

for fold in range(K):

    model = lgb.LGBMRegressor(
                objective = qwk_objective,
                metrics = 'None',
                learning_rate = 0.05,
                max_depth = 5,
                num_leaves = 10,
                colsample_bytree=0.3,
                reg_alpha = 0.7,
                reg_lambda = 0.1,
                n_estimators=700,
                random_state=42,
                extra_trees=True,
                class_weight='balanced',
                verbosity = - 1)

    a, b = calculate_qwk_parameters(train_feats[train_feats['fold'] != fold]['score'])
    X_train = train_feats[train_feats['fold'] != fold].drop(columns=drop_columns+target)
    y_train = train_feats[train_feats['fold'] != fold]['score'] - a

    X_eval = train_feats[train_feats['fold'] == fold].drop(columns=drop_columns+target)
    y_eval = train_feats[train_feats['fold'] == fold]['score'] - a

    print(f"\nTraining fold {fold} with a: {a}, b: {b}")

    lgb_model = model.fit(
        X_train, y_train,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, y_train), (X_eval, y_eval)],
        eval_metric=calculate_quadratic_weighted_kappa,
        callbacks=training_callbacks
    )

    lgb_models.append(lgb_model)

    pred = model.predict(X_eval)
    actuals.extend(y_eval + a)
    predictions.extend(np.round(pred + a, 0))

validation_score = cohen_kappa_score(actuals, predictions, weights="quadratic")
print(f"Validation score: {validation_score}")
print(f"Training time: {time.time() - t0}")


Training fold 0 with a: 2.9484, b: 1.0917
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.696419	valid's QWK: 0.697293
[50]	train's QWK: 0.736526	valid's QWK: 0.73559
[75]	train's QWK: 0.762882	valid's QWK: 0.759497
[100]	train's QWK: 0.778918	valid's QWK: 0.769733
[125]	train's QWK: 0.791712	valid's QWK: 0.776813
[150]	train's QWK: 0.800444	valid's QWK: 0.784007
[175]	train's QWK: 0.807804	valid's QWK: 0.790063
[200]	train's QWK: 0.812795	valid's QWK: 0.793128
[225]	train's QWK: 0.817882	valid's QWK: 0.796521
[250]	train's QWK: 0.82262	valid's QWK: 0.797341
[275]	train's QWK: 0.825257	valid's QWK: 0.797368
[300]	train's QWK: 0.82938	valid's QWK: 0.797457
[325]	train's QWK: 0.833677	valid's QWK: 0.798217
[350]	train's QWK: 0.835895	valid's QWK: 0.798551
[375]	train's QWK: 0.83851	valid's QWK: 0.798541
[400]	train's QWK: 0.841868	valid's QWK: 0.799175
[425]	train's QWK: 0.844072	valid's QWK: 0.8011

**Kết luận:** Nhóm nhận thấy mô hình **Light Gradient-Boosting Machine** cho ra kết quả tốt nhất, nên sẽ sử dụng mô hình này để dự đoán kết quả cuối cùng.

### **3. Dự đoán kết quả**

In [ ]:
test_predictions = []
drop_columns = ['essay_id']

for fold, model in enumerate(lgb_models):
    X_test = test_feats.drop(columns=drop_columns)
    fold_predictions = model.predict(X_test) + a
    test_predictions.append(fold_predictions)

for i, fold_predictions in enumerate(test_predictions):
    test_feats[f"score_pred_{i}"] = fold_predictions

test_feats["score"] = np.round(test_feats[[f"score_pred_{fold}" for fold in range(K)]].mean(axis=1), 0).astype('int32')

test_feats[['essay_id', 'score']].head()

,essay_id,score
0,000d118,2
1,000fe60,3
2,001ab80,5
